# Project: Apply Lightweight Fine-Tuning to a Foundation Model

In this project, have used the Emotions dataset [https://huggingface.co/datasets/dair-ai/emotion]

The dataset contains:
- Text column which has statements that need to be classified as one of the emotions
- Label column which has labels describing the emotion as sadness (0), joy (1), love (2), anger (3), fear (4), surprise (5)  

Initially the foundation model BERT sentiment classifier (DistilBERT) will be used, without finetuning the base model parameters and then its performance will be evaluated. Then, the model will be trained using PEFT approach to finetune the parameters and the performance will be evaluated. Finally, we will compare the performance of the model without parameter finetuning and with parameter efficient finetuning.

### Importing Libraries and Loading the dataset

In [1]:
# Importing necessary libraries
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
from datasets import load_dataset
from peft import LoraConfig, get_peft_model

In [2]:
# Load the split configuration of the Emotion dataset
dataset = load_dataset("emotion", "split")

# View the dataset characteristics
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})


### Preprocessing the dataset

In [3]:
# Load a pre-trained tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Inspect the tokenized dataset
print(tokenized_dataset["train"][0])

/Users/sauravverma/anaconda3/envs/udacitygenai/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


{'text': 'i didnt feel humiliated', 'label': 0, 'input_ids': [101, 1045, 2134, 2102, 2514, 26608, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}


### Load and Setup the model without finetuning the base model parameters

In [4]:
# Load the pre-trained model for sequence classification
model_nofinetuning = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=6,
    id2label={0: "sadness", 1: "joy", 2: "love", 3: "anger", 4: "fear", 5: "surprise"},
    label2id={"sadness": 0, "joy": 1, "love": 2, "anger": 3, "fear": 4, "surprise": 5},
)

#Freeze all the parameters of base model
for param in model_nofinetuning.base_model.parameters():
    param.requires_grad = False

model_nofinetuning.classifier

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Linear(in_features=768, out_features=6, bias=True)

In [5]:
print(model_nofinetuning)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

### Train the Model

In [6]:
#Accuracy Metric to be supplied to trainer
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

In [7]:
# Create the Trainer
trainer_nofinetuning = Trainer(
    model=model_nofinetuning,
    args=TrainingArguments(
        output_dir="./data/emotions",
        learning_rate=2e-5,
        per_device_train_batch_size=64,
        per_device_eval_batch_size=64,
        evaluation_strategy="epoch",
        save_strategy = "epoch",
        num_train_epochs=3,
        weight_decay=0.01,
        load_best_model_at_end=True,
        logging_dir='.data/emotions/logs_no_finetuning',
        logging_steps=100,
        ),
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

# Train the model
trainer_nofinetuning.train()

/Users/sauravverma/anaconda3/envs/udacitygenai/lib/python3.11/site-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,1.574200,1.548101,0.454000
2,1.524100,1.514742,0.485000
3,1.504200,1.503843,0.486500


TrainOutput(global_step=750, training_loss=1.5450640665690105, metrics={'train_runtime': 51.5627, 'train_samples_per_second': 930.905, 'train_steps_per_second': 14.545, 'total_flos': 703485182771712.0, 'train_loss': 1.5450640665690105, 'epoch': 3.0})

### Evaluate the Model without Finetuning and visualize its performance

In [8]:
trainer_nofinetuning.evaluate()

{'eval_loss': 1.503842830657959,
 'eval_accuracy': 0.4865,
 'eval_runtime': 1.4723,
 'eval_samples_per_second': 1358.388,
 'eval_steps_per_second': 21.734,
 'epoch': 3.0}

In [10]:
# Make a dataframe with the predictions and the text and the labels

results_nofinetuning = trainer_nofinetuning.predict(tokenized_dataset["test"])

# Get the predicted labels
predicted_labels_nofinetuning = np.argmax(results_nofinetuning.predictions, axis=1)

# Get the text of the examples
texts = [item["text"] for item in tokenized_dataset["test"]]

# Create a DataFrame
df = pd.DataFrame({
    "Text": texts,
    "True Label": results_nofinetuning.label_ids,
    "Predicted Label": predicted_labels_nofinetuning
})

# Map label indices to actual emotions
label_map = {0: "sadness", 1: "joy", 2: "love", 3: "anger", 4: "fear", 5: "surprise"}
df["True Label"] = df["True Label"].map(label_map)
df["Predicted Label"] = df["Predicted Label"].map(label_map)

df.head(10)

,Text,True Label,Predicted Label
0,im feeling rather rotten so im not very ambiti...,sadness,joy
1,im updating my blog because i feel shitty,sadness,sadness
2,i never make her separate from me because i do...,sadness,sadness
3,i left with my bouquet of red and yellow tulip...,joy,joy
4,i was feeling a little vain when i did this one,sadness,joy
5,i cant walk into a shop anywhere where i do no...,fear,sadness
6,i felt anger when at the end of a telephone call,anger,sadness
7,i explain why i clung to a relationship with a...,joy,joy
8,i like to have the same breathless feeling as ...,joy,joy
9,i jest i feel grumpy tired and pre menstrual w...,anger,joy


### Prepare for Parameter Efficient Fine Tuning of the Model

In [11]:
# # Load the pre-trained model for sequence classification
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=6,
    id2label={0: "sadness", 1: "joy", 2: "love", 3: "anger", 4: "fear", 5: "surprise"},
    label2id={"sadness": 0, "joy": 1, "love": 2, "anger": 3, "fear": 4, "surprise": 5},
)

/Users/sauravverma/anaconda3/envs/udacitygenai/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
# Apply LoRA for parameter-efficient fine-tuning
config = LoraConfig(
    r=4,               # Rank of the adaptation matrix
    lora_alpha=32,     # Alpha scaling factor
    target_modules=["q_lin", "k_lin", "v_lin", "out_lin"],  # Adjust target modules for DistilBERT
    lora_dropout=0.1   # Dropout rate
)
model = get_peft_model(model, config)
model.print_trainable_parameters()

trainable params: 147,456 || all params: 67,105,542 || trainable%: 0.2197


In [13]:
# Unfreeze the parameters in the target modules
for name, param in model.named_parameters():
    if any(target in name for target in config.target_modules):
        param.requires_grad = True

In [14]:
labels = tokenized_dataset["train"].features["label"].names

### Train the model with PEFT

In [15]:
# Create the Trainer for the second training (with PEFT)
trainer_with_peft = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir="./data/emotions_with_peft",
        learning_rate=2e-5,
        label_names=['labels'],
        per_device_train_batch_size=64,
        per_device_eval_batch_size=64,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=3,
        weight_decay=0.01,
        load_best_model_at_end=True,
        logging_dir='./data/emotions_with_peft/logs_with_peft',
        logging_steps=100,
        ),
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

# Train the model with PEFT
trainer_with_peft.train()

/Users/sauravverma/anaconda3/envs/udacitygenai/lib/python3.11/site-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,1.203500,0.921001,0.732500
2,0.751400,0.663432,0.800000
3,0.655300,0.600001,0.834500


/Users/sauravverma/anaconda3/envs/udacitygenai/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/Users/sauravverma/anaconda3/envs/udacitygenai/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


TrainOutput(global_step=750, training_loss=0.9353773651123047, metrics={'train_runtime': 174.5644, 'train_samples_per_second': 274.97, 'train_steps_per_second': 4.296, 'total_flos': 705890645475840.0, 'train_loss': 0.9353773651123047, 'epoch': 3.0})

### Evaluate the Model with PEFT and visualize its performance

In [16]:
trainer_with_peft.evaluate()

{'eval_loss': 0.600001335144043,
 'eval_accuracy': 0.8345,
 'eval_runtime': 1.8378,
 'eval_samples_per_second': 1088.275,
 'eval_steps_per_second': 17.412,
 'epoch': 3.0}

In [17]:
# Make a dataframe with the predictions and the text and the labels

results_peft = trainer_with_peft.predict(tokenized_dataset["test"])

# Get the predicted labels
predicted_labels = np.argmax(results_peft.predictions, axis=1)

# Get the text of the examples
texts = [item["text"] for item in tokenized_dataset["test"]]

# Create a DataFrame
df = pd.DataFrame({
    "Text": texts,
    "True Label": results_peft.label_ids,
    "Predicted Label": predicted_labels
})

# Map label indices to actual emotions
label_map = {0: "sadness", 1: "joy", 2: "love", 3: "anger", 4: "fear", 5: "surprise"}
df["True Label"] = df["True Label"].map(label_map)
df["Predicted Label"] = df["Predicted Label"].map(label_map)

df.head(10)

,Text,True Label,Predicted Label
0,im feeling rather rotten so im not very ambiti...,sadness,sadness
1,im updating my blog because i feel shitty,sadness,sadness
2,i never make her separate from me because i do...,sadness,sadness
3,i left with my bouquet of red and yellow tulip...,joy,joy
4,i was feeling a little vain when i did this one,sadness,sadness
5,i cant walk into a shop anywhere where i do no...,fear,fear
6,i felt anger when at the end of a telephone call,anger,anger
7,i explain why i clung to a relationship with a...,joy,sadness
8,i like to have the same breathless feeling as ...,joy,joy
9,i jest i feel grumpy tired and pre menstrual w...,anger,anger


### Conclusion

We can clearly see that the model with performance efficient fine tuning has a much higher accuracy (over 83%) compared to the model with no finetuning (over 48%)